In [1]:
'''
logic

get unique game_ids in the pbp
get the gamelog for the most recent nba games this season

see what games you dont have

for those games get the pbp data
merge it all together and upload it to database


for all the games you download

create the features
merge with elo scores
use the model and predict on them 
save them to database 

'''

'''
change that 
source fo truth on game_log'''

from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['NBA']

db.list_collection_names()



import pandas as pd
import numpy as np
from nba_api.stats.endpoints import playbyplayv2, leaguegamelog, leaguegamefinder
from preprocess import feature_engineer
from nba_helper import read_url_to_csv, elo_url
from time import sleep

In [88]:
# get current game log

gl = pd.DataFrame.from_records(db.game_log.find())

In [17]:
lgl = leaguegamelog.LeagueGameLog().get_data_frames()[0].astype({'GAME_ID':int})

In [89]:
games_to_update = set(lgl['GAME_ID']).difference(set(gl['GAME_ID']))

In [91]:
if games_to_update:
    print(5)
else:
    print(4)

4


In [73]:
games_to_update_df = lgl[(lgl['GAME_ID'].isin(games_to_update))&\
   (~lgl['MATCHUP'].str.contains('@'))]

In [74]:
db.game_log.insert_many(games_to_update_df.to_dict('records'))

In [33]:
games_to_update_df[['Home', 'Away']] = games_to_update_df['MATCHUP'].str.split('vs.', expand=True)
games_to_update_df.loc[:, 'Home'] = games_to_update_df['Home'].str.strip()
games_to_update_df.loc[:, 'home_team_win'] = games_to_update_df['WL'].replace({'W':1, 'L':0})

In [36]:
elo = read_url_to_csv(elo_url)
elo = elo[elo['date'] > '2012-01-01']

elo.loc[:, 'elo_difference'] = np.abs(elo['elo1_pre'] - elo['elo2_pre'])

elo = elo[['date', 'team1', 'elo1_pre', 'elo2_pre', 'elo_difference']]

elo['team1'] = elo['team1'].replace({'BRK':'BKN',
                                     'PHO':'PHX',
                                    'CHO':'CHA',})

In [40]:
df = games_to_update_df.merge(elo, 
                             left_on=['GAME_DATE', 'Home'], 
                             right_on = ['date', 'team1'])[['GAME_ID', 'elo1_pre', 'elo2_pre', 
                                                           'elo_difference', 'home_team_win']]

In [41]:
df

,GAME_ID,elo1_pre,elo2_pre,elo_difference,home_team_win
0,22100584,1527.509399,1695.642200,168.132802,1
1,22100583,1576.222215,1580.527158,4.304943,0
2,22100585,1585.590869,1465.948081,119.642788,1
3,22100587,1335.622199,1498.299968,162.677769,0
4,22100586,1324.919635,1562.153280,237.233645,0
...,...,...,...,...,...
56,22100637,1555.443668,1634.072143,78.628474,0
57,22100636,1612.779570,1463.134333,149.645237,1
58,22100638,1475.210747,1597.842734,122.631987,0
59,22100639,1628.497290,1564.133002,64.364288,0


In [44]:
pbps = []
for g_id in df['GAME_ID'].unique():
    pbps.append(playbyplayv2.PlayByPlayV2(f'00{g_id}').get_data_frames()[0])
    sleep(0.5)

In [50]:
pbp_df = pd.concat(pbps).astype({'GAME_ID':int})

pbp_df_prepped = feature_engineer(pbp_df)

In [55]:
final_df = pbp_df_prepped.merge(df, on=['GAME_ID'])

In [57]:
wEloCols = ['home_poss', 'diff', 'time_remaining', 'OT_ind', 'elo1_pre', 'elo2_pre']

wOEloCols = ['home_poss', 'diff', 'time_remaining', 'OT_ind']

In [63]:
import tensorflow as tf
model = tf.keras.models.load_model('../Models/TF_model_w_elo.h5')
model_wO_elo = tf.keras.models.load_model('../Models/TF_model_wO_elo.h5')

In [64]:
final_df.loc[:, 'preds_w_elo'] = model.predict_on_batch(final_df[wEloCols])
final_df.loc[:, 'preds_wO_elo'] = model_wO_elo.predict_on_batch(final_df[wOEloCols])

In [65]:
final_df

,GAME_ID,home_poss,diff,time_remaining,OT_ind,elo1_pre,elo2_pre,elo_difference,home_team_win,preds_w_elo,preds_wO_elo
0,22100584,0,0,2880,0,1527.509399,1695.642200,168.132802,1,0.375489,0.607601
1,22100584,1,0,2880,0,1527.509399,1695.642200,168.132802,1,0.374682,0.610178
2,22100584,0,0,2857,0,1527.509399,1695.642200,168.132802,1,0.376107,0.606385
3,22100584,0,0,2851,0,1527.509399,1695.642200,168.132802,1,0.376269,0.606068
4,22100584,1,0,2849,0,1527.509399,1695.642200,168.132802,1,0.375515,0.608544
...,...,...,...,...,...,...,...,...,...,...,...
28217,22100640,1,11,20,0,1370.181727,1311.455836,58.725891,1,0.997234,0.999973
28218,22100640,1,12,20,0,1370.181727,1311.455836,58.725891,1,0.998251,0.999991
28219,22100640,0,12,14,0,1370.181727,1311.455836,58.725891,1,0.998372,0.999992
28220,22100640,1,12,11,0,1370.181727,1311.455836,58.725891,1,0.998355,0.999995


In [66]:
db.list_collection_names()

['historical_raptor',
 'player2k',
 'game_log',
 'shot_chart',
 'historical_pbp_modelled',
 'historical_pbp']

In [67]:
db.historical_pbp_modelled.insert_many(final_df.to_dict('records'))

In [75]:
game_date = '2022-01-07'
data = db.game_log.find({"GAME_DATE":game_date})
d = pd.DataFrame.from_records(data)

In [76]:
d

,_id,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,61e2e16e79a1a87533b12f3f,22021,1610612761,TOR,Toronto Raptors,22100584,2022-01-07,TOR vs. UTA,W,240,45,100,0.450,12,41,0.293,20,25,0.800,19,29,48,26,11,7,9,17,122,14,1
1,61e2e16e79a1a87533b12f40,22021,1610612751,BKN,Brooklyn Nets,22100583,2022-01-07,BKN vs. MIL,L,240,42,92,0.457,6,27,0.222,19,20,0.950,14,35,49,23,8,5,11,16,109,-12,1
2,61e2e16e79a1a87533b12f41,22021,1610612741,CHI,Chicago Bulls,22100585,2022-01-07,CHI vs. WAS,W,240,48,91,0.527,15,33,0.455,19,21,0.905,10,31,41,34,11,3,15,21,130,8,1
3,61e2e16e79a1a87533b12f42,22021,1610612760,OKC,Oklahoma City Thunder,22100587,2022-01-07,OKC vs. MIN,L,240,36,89,0.404,15,44,0.341,18,23,0.783,13,32,45,26,10,4,20,23,105,-30,1
4,61e2e16e79a1a87533b12f43,22021,1610612745,HOU,Houston Rockets,22100586,2022-01-07,HOU vs. DAL,L,240,37,87,0.425,12,43,0.279,20,29,0.690,7,33,40,17,6,3,13,20,106,-24,1
5,61e2e16e79a1a87533b12f44,22021,1610612743,DEN,Denver Nuggets,22100588,2022-01-07,DEN vs. SAC,W,240,47,85,0.553,12,34,0.353,15,24,0.625,5,36,41,32,5,7,8,20,121,10,1
6,61e2e16e79a1a87533b12f45,22021,1610612755,PHI,Philadelphia 76ers,22100582,2022-01-07,PHI vs. SAS,W,240,45,87,0.517,8,23,0.348,21,27,0.778,12,37,49,28,7,4,13,18,119,19,1
7,61e2e16e79a1a87533b12f46,22021,1610612747,LAL,Los Angeles Lakers,22100589,2022-01-07,LAL vs. ATL,W,240,50,101,0.495,17,41,0.415,17,19,0.895,8,39,47,37,14,4,8,26,134,16,1
8,61e2e16e79a1a87533b12f47,22021,1610612757,POR,Portland Trail Blazers,22100590,2022-01-07,POR vs. CLE,L,240,37,92,0.402,16,35,0.457,11,19,0.579,16,35,51,22,8,5,12,19,101,-13,1


In [77]:
from apscheduler.schedulers.background import BackgroundScheduler

In [97]:
from pytz import utc
scheduler = BackgroundScheduler()
scheduler.configure(timezone=utc)
def myfunc():
    print('t')
    return True

job = scheduler.add_job(myfunc, 'interval', seconds=2, id='test_job')
scheduler.start()

t
t
t
t
t
t
t
t
t
t


In [98]:
scheduler.shutdown()